In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movies data processing").getOrCreate()

In [2]:
from pyspark.sql.types import  *
from pyspark.sql.functions import col

In [3]:
import numpy as np

In [4]:
movies = spark.read.csv("datasets/movies_metadata.csv", header = True) 

In [5]:
movies.dtypes

[('adult', 'string'),
 ('belongs_to_collection', 'string'),
 ('budget', 'string'),
 ('genres', 'string'),
 ('homepage', 'string'),
 ('id', 'string'),
 ('imdb_id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('poster_path', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('runtime', 'string'),
 ('spoken_languages', 'string'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('video', 'string'),
 ('vote_average', 'string'),
 ('vote_count', 'string')]

In [6]:
movies.select(["genres"]).show(10, truncate = False)

+-----------------------------------------------------------------------------------------------------------------------------+
|genres                                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------+
|[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]                             |
|[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]                            |
|[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]                                                             |
|[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]                                |
|[{'id': 35, 'name': 'Comedy'}]                                                                         

In [7]:
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.functions import udf

def to_struct_array(col):
    return to_struct(split(regexp_replace(col, "\[|\]", ""), "\}, \{"))


to_struct_array_udf = udf(lambda z: to_struct_array(z), ArrayType(StringType()))
    

In [9]:
movies = movies.withColumn("adult", (movies.adult).cast("Boolean"))\
         .withColumn("budget", (movies.budget).cast("Integer"))\
         .withColumn("genres", split(regexp_replace(regexp_replace(movies.genres, "\}, \{", "\}; \{"), "\[|\]", ""), ";"))
    
#  .withColumn("genres", split(regexp_replace(movies.genres, "\[|\]", ""), ","))
         
#.withColumn("genres", split(regexp_replace(movies.genres, "\[|\]", ""), ","))

In [10]:
movies.select(["genres"]).show(10, truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------+
|genres                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------+
|[{'id': 16, 'name': 'Animation'},  {'id': 35, 'name': 'Comedy'},  {'id': 10751, 'name': 'Family'}]                              |
|[{'id': 12, 'name': 'Adventure'},  {'id': 14, 'name': 'Fantasy'},  {'id': 10751, 'name': 'Family'}]                             |
|[{'id': 10749, 'name': 'Romance'},  {'id': 35, 'name': 'Comedy'}]                                                               |
|[{'id': 35, 'name': 'Comedy'},  {'id': 18, 'name': 'Drama'},  {'id': 10749, 'name': 'Romance'}]                                 |
|[{'id': 35, 'name': 'Comedy'}]                                                    

In [ ]:
movies.dtypes

In [ ]:
from pyspark.sql.functions import split, regexp_replace

movies = movies.withColumn(
    "genres",
    split(regexp_replace(movies.genres, r"(^\[\[\[)|(\]\]\]$)", ""), ", ")
)
df2.show()



In [ ]:
movies.dtypes

In [ ]:
movies.printSchema()

In [ ]:
import pandas as pd

df = pd.read_csv("datasets_obli/movies_metadata.csv")

In [ ]:
del df['homepage']


In [ ]:
collection = df.head(1)['belongs_to_collection']